### Import Data

In [1]:
import json
import requests

# url = "https://fantasy.premierleague.com/drf/bootstrap-static"
url = 'https://fantasy.premierleague.com/api/bootstrap-static/'

def get_json_data(url):
    response = requests.get(url)
    jobs = response.json()
    return jobs

#filter out players with less than 330
all_players = get_json_data(url)['elements']
all_teams = get_json_data(url)['teams']
positions = get_json_data(url)['element_types']

In [2]:
def attach_team_and_position(main_players, all_teams, positons):
    players = main_players.copy()
    for team in all_teams:
        for player in players:
            if team['code'] == player['team_code']:
                player['team_name'] = team['name']
    for position in positions:
        for player in players:
            if position['id'] == player['element_type']:
                player['position'] = position['singular_name']
    for player in players:
        if player['status'] == "a":
            player['status'] = "available"
        elif player['status'] == "d":
            player['status'] = "doubtful"
        elif player['status'] == "i":
            player['status'] = "injured"
        elif player['status'] == "s":
            player['status'] = "suspended"
    return players


#### Temporarily took out the "main_players" filter because no players have played 330 minutes ####

# player_positions_teams = attach_team_and_position(main_players, all_teams_new, positions)
player_positions_teams = attach_team_and_position(all_players, all_teams, positions)


def final_players_list(player_positions_teams):
    players = player_positions_teams.copy()
    empty = []
    for player in players:
        stats = dict(team_code = player['team_code'],
        team_name = player['team_name'],
        name = player['first_name'] + " " + player['second_name'],
        status = player['status'],
        position = player['position'],
        cost = (player['now_cost'])/10,
        total_points = player['total_points'],
        roi = round((player['total_points'] / player['now_cost'])*10, 2),
        pts_per_90min = round((player['total_points'] / (player['minutes'] + 1))*90, 2),
        bonus = player['bonus'],
        red_cards = player['red_cards'],
        minutes = player['minutes'],
        transfers_out = player['transfers_out'],
        transfers_in = player['transfers_in'])
        empty.append(stats)
    return empty

In [3]:
final = final_players_list(player_positions_teams)

In [4]:
final[0]

{'team_code': 3,
 'team_name': 'Arsenal',
 'name': 'Shkodran Mustafi',
 'status': 'available',
 'position': 'Defender',
 'cost': 5.5,
 'total_points': 80,
 'roi': 14.55,
 'pts_per_90min': 2.76,
 'bonus': 5,
 'red_cards': 0,
 'minutes': 2611,
 'transfers_out': 0,
 'transfers_in': 0}

In [5]:
from sqlalchemy import *
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship

Base = declarative_base()

class Player(Base):
    __tablename__ = "players"
    id = Column(Integer, primary_key = True)
    team_id = Column(Integer, ForeignKey('teams.id'))
    team = relationship('Team', back_populates = 'players')
    team_code = Column(Integer)
    name = Column(String)
    position = Column(String)
    cost = Column(Float)
    total_points = Column(Integer)
    roi = Column(Float)
    pts_per_90min = Column(Float)
    bonus = Column(Integer)
    red_cards = Column(Integer)
    minutes = Column(Integer)
    status = Column(String)
    transfers_out = Column(Integer)
    transfers_in = Column(Integer)

class Team(Base):
    __tablename__ = "teams"
    id = Column(Integer, primary_key = True)
    # table_id = Column(Integer, Foreign_Key('epl_table.id'))
    code = Column(Integer)
    position = Column(Integer)
    strength = Column(Integer)
    name = Column(String)
    points = Column(Integer)
    player_points = Column(Integer)
    players = relationship('Player', back_populates = 'team')

In [6]:
all_teams[0]

{'code': 3,
 'draw': 0,
 'form': None,
 'id': 1,
 'loss': 0,
 'name': 'Arsenal',
 'played': 0,
 'points': 0,
 'position': 0,
 'short_name': 'ARS',
 'strength': 4,
 'team_division': None,
 'unavailable': False,
 'win': 0,
 'strength_overall_home': 1250,
 'strength_overall_away': 1330,
 'strength_attack_home': 1240,
 'strength_attack_away': 1270,
 'strength_defence_home': 1290,
 'strength_defence_away': 1330}

In [7]:
final[0]

{'team_code': 3,
 'team_name': 'Arsenal',
 'name': 'Shkodran Mustafi',
 'status': 'available',
 'position': 'Defender',
 'cost': 5.5,
 'total_points': 80,
 'roi': 14.55,
 'pts_per_90min': 2.76,
 'bonus': 5,
 'red_cards': 0,
 'minutes': 2611,
 'transfers_out': 0,
 'transfers_in': 0}

In [12]:
import pandas as pd

In [16]:
player_df = pd.DataFrame(final)

In [18]:
player_df.head()

,bonus,cost,minutes,name,position,pts_per_90min,red_cards,roi,status,team_code,team_name,total_points,transfers_in,transfers_out
0,5,5.5,2611,Shkodran Mustafi,Defender,2.76,0,14.55,available,3,Arsenal,80,0,0
1,5,5.5,1532,Héctor Bellerín,Defender,3.52,0,10.91,injured,3,Arsenal,60,0,0
2,10,5.5,1886,Sead Kolasinac,Defender,3.86,0,14.73,available,3,Arsenal,81,0,0
3,3,5.0,983,Ainsley Maitland-Niles,Defender,3.11,1,6.80,available,3,Arsenal,34,0,0
4,4,5.0,2198,Sokratis Papastathopoulos,Defender,2.62,0,12.80,available,3,Arsenal,64,0,0


In [19]:
team_df = pd.DataFrame(all_teams)

In [20]:
team_df.head()

,code,draw,form,id,loss,name,played,points,position,short_name,strength,strength_attack_away,strength_attack_home,strength_defence_away,strength_defence_home,strength_overall_away,strength_overall_home,team_division,unavailable,win
0,3,0,None,1,0,Arsenal,0,0,0,ARS,4,1270,1240,1330,1290,1330,1250,None,False,0
1,7,0,None,2,0,Aston Villa,0,0,0,AVL,2,1050,1050,1070,1030,1050,990,None,False,0
2,91,0,None,3,0,Bournemouth,0,0,0,BOU,3,1100,1040,1170,1120,1140,1050,None,False,0
3,36,0,None,4,0,Brighton,0,0,0,BHA,2,1140,1040,1070,1050,1070,1000,None,False,0
4,90,0,None,5,0,Burnley,0,0,0,BUR,3,1030,990,1070,1060,1090,1070,None,False,0


In [ ]:
def roi_top_players():
    return session.query(Player).order_by(Player.roi.desc()).all()

def pts_per_90m_top_players():
    return session.query(Player).order_by(Player.pts_per_90min.desc()).all()


In [ ]:
def build_team_by_roi(budget = 100, count_limit = 3, gk = 2, df = 5, md = 5, fwd = 3):
    money_team = []
    budget = budget
    injured = players_by_status('injured')
    positions = {'Goalkeeper': gk, 'Defender': df, 'Midfielder': md, 'Forward': fwd}
    teams = { team:3 for team in team_list() }
    for player in points_top_players():
            if len(money_team) < count_limit and player not in injured and budget >= player.cost and positions[player.position] > 0 and teams[player.team] > 0:
                money_team.append(player)
                budget -= player.cost
                positions[player.position] = positions[player.position] - 1
                teams[player.team] = teams[player.team] -1
            else:
                for player in roi_top_players():
                    if player not in money_team and budget >= player.cost and positions[player.position] > 0 and teams[player.team] > 0:
                        money_team.append(player)
                        budget -= player.cost
                        positions[player.position] = positions[player.position] - 1
                        teams[player.team] = teams[player.team] -1
    final_team = [(item.name, item.position, item.cost) for item in money_team]
    total_points = sum([item.total_points for item in money_team])
    print('Remaining Budget: ' + str(round(budget, 2)))
    print('Your AI has picked the following team:')
    print('GK: '), print([(item[0], item[2]) for item in final_team if item[1] == "Goalkeeper"])
    print('DF: '), print([(item[0], item[2]) for item in final_team if item[1] == "Defender"])
    print('MD: '), print([(item[0], item[2])  for item in final_team if item[1] == "Midfielder"])
    print('FWD: '), print([(item[0], item[2])  for item in final_team if item[1] == "Forward"])
    print('Total Fantasy Points: ' + str(total_points))
    return money_team